In [1]:
from IPython.display import HTML

HTML('''<script>

code_show=true; 

function code_toggle() {
if (code_show){
$('div.input').hide();
    if (document.getElementById('input') !== null)
        {document.getElementById('input').value = 'Display code in notebook';}
} else {
$('div.input').not(':first').show();
document.getElementById('input').value = 'Hide code in notebook';
}
code_show = !code_show
}

$( document ).ready(code_toggle);
$('div.cell.code_cell.rendered.selected').find('div.input').hide();



</script>

<form id="form" action="javascript:code_toggle()">
    <input id="input" type="submit" value="Display code in notebook">
</form>




''')

In [ ]:
import dask.dataframe as dd
import os
import pandas as pd
import seaborn as sns
import numpy as np
from dask.distributed import Client
import plotly.graph_objects as go
import geopandas as gpd
import plotly.express as px
import json
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import hvplot.pandas
import sys
from io import StringIO
import requests
import xarray as xr
import warnings

pd.set_option('display.max_rows', 2000)
warnings.filterwarnings("ignore")

# Local functions
import sefm.backends.climate.storm_typing as clim
import sefm.utils.hydrotools as ht

# 1. Introduction

### 1.1 Usage

Storm typing classifies storms events into classes that are nearly homogeneous with respects to their generating phenomenon. This allows to evaluate more homogeunous distributions for each storm type when computing precipitation-frequency analysis rather than a traditionel mixed distribution with all the storms. Homogeneity is an assumption when computing frequency analysis which makes storm typing all the more important.

### 1.2 Storm types

Description ...

### 1.3 Database of Daily Storm Types (DDST)

Description ...

## 2. Calculations

Introduction...

![alt text](https://s3.us-east-2.wasabisys.com/analytics-store/img/AMS.png?raw=true)

## Intrants

Définition d'une région spatiale afin de limiter l'extraction des données météos

In [ ]:
from IPython.display import Markdown as md

md("Définition d'une région spatiale afin de limiter l'extraction des données météos = %s"%(str(latlngbox)))

In [ ]:
latlngbox = [-82, -74, 44.5, 49]



Définition des caracrétistiques du stockage des données de stations météorologiques (GHCN-D)

In [ ]:
bucket = 's3://ghcnd-can-us-ne'
storage_options={'anon': True,
                 "client_kwargs": {'endpoint_url': 'https://s3.us-east-2.wasabisys.com'}}

## Client Dask 

Le client Dask permet de paralléliser l'ensemble du code ci-après et d'effectuer le calcul par lazy loading. Lorsqu'un calcul est lancé, celui-ci peut être suivi en temps réel via le tableau de bord (dashboard) dont l'adresse est affichée ci-dessous.

In [ ]:
client = Client()
client

## Importation des stations météorologiques (GHCN-D)

In [ ]:
st = clim.Stations(metadata_bucket=os.path.join(bucket,'ghcdn_stations.csv'),
                   storage_options=storage_options)

### Importation des métadonnées

Les métadonnées sont filtrés via les limites géographiques définies dans la variable <em>latlngbox</em>
Notons que les métadonnées peuvent également être filtrées à l'aide du ou des numéros de stations.

Exemple: *st.read_metadata(station_names=['CA007038080',...])*

In [ ]:
st.read_metadata(latlngbox=latlngbox)

### Importation des données

Les données sont acquises via *lazy loading* à partir du format parquet afin de profiter de l'indexation des séries temporelles et ramener les données plus rapidement pour réaliser le calcul

In [ ]:
df = st.read_parquet(os.path.join(bucket,'data/parquet/data.parquet'),
                     element='PRCP',
                     storage_options=storage_options)
df['value'] = df['value']/10.0 # Pour convertir vers mm

## Transformation des données

### Pivot des données

Les données sont pivotées à l'aide de l'index (numéro de station) afin d'obtenir une colonne de séries temporelles par station. Pour avoir suffisamment de données de qualité, seules les données après 1900 sont retenues

In [ ]:
df_pivot = df.pivot_table(index='date', columns='id', values='value')
df_pivot.index = pd.to_datetime(df_pivot.index)

df_pivot = df_pivot[df_pivot.index.year>=1900]
df_pivot

In [ ]:
# FROM THIS POINT ON, THIS IS A WORK IN PROGRESS

In [ ]:
gdf_outaouais = gpd.read_file('/home/slanglois/PycharmProjects/DEMv2/data/Bassins Outaouais HSAMI.geojson')
ddst.visualisation_stations(gdf_outaouais, 
                            df_stations,
                            latlngbox)

In [ ]:
store = os.path.join(data_path,
                     'data/reanalysis/CIRES_20_Century_Reanalysis/3h/single-levels/zarr')
ds = xr.open_zarr(store=store,
                  consolidated=True)

In [ ]:
ds

In [ ]:
zones_path = '/media/slanglois/ssd-2tb/sefm/data/gis/AMS.shp'
gdf = gpd.read_file(zones_path)
gdf

In [ ]:
df_cape = ht.clip_polygon_to_dataframe(dataset=ds,
                                       geodataframe=gdf.sort_values(by='id'),
                                       geodf_index_column='id',
                                       variable='cape',  
                                       aggregation='mean',
                                       resample_time='1D', # originalement au pas de temps horaire
                                       from_tz='UTC',
                                       to_tz='America/Montreal',
                                       latlng_names=['latitude',
                                                    'longitude']
                                       )

df_cape.columns = ['Cape1', 'Cape2', 'Cape3', 'Cape4']

In [ ]:
df_pr_wtr=ht.clip_polygon_to_dataframe(dataset=ds,
                                       geodataframe=gdf.sort_values(by='id'),
                                       geodf_index_column='id',
                                       variable='pr_wtr',  
                                       aggregation='sum',
                                       resample_time='1D', # originalement au pas de temps horaire
                                       from_tz='UTC',
                                       to_tz='America/Montreal',
                                       latlng_names=['latitude',
                                                    'longitude']
                                       )
df_pr_wtr.columns = ['Pw1', 'Pw2', 'Pw3', 'Pw4']

In [ ]:
df_stations_complet = df_stations[df_stations['id'].isin(df['id'].unique())]
gdf_stations = gpd.GeoDataFrame(df_stations_complet, geometry=gpd.points_from_xy(df_stations_complet.longitude,
                                                                                 df_stations_complet.latitude))

In [ ]:
df2 = pd.read_csv('/media/slanglois/ssd-2tb/sefm/data/observations/HQP/liste_stations_HQP.csv')

In [ ]:
gdf_stations2 = gpd.GeoDataFrame(df2, geometry=gpd.points_from_xy(df2.xcoord, df2.ycoord))

In [ ]:
ams4

In [ ]:
ams1=gdf_stations2[gdf_stations2.within(gdf.iloc[0].geometry)].NOM_DETAILLE
ams2=gdf_stations2[gdf_stations2.within(gdf.iloc[1].geometry)].NOM_DETAILLE
ams3=gdf_stations2[gdf_stations2.within(gdf.iloc[2].geometry)].NOM_DETAILLE
ams4=gdf_stations2[gdf_stations2.within(gdf.iloc[3].geometry)].NOM_DETAILLE

In [ ]:
ams1=gdf_stations[gdf_stations.within(gdf.iloc[0].geometry)].id
ams2=gdf_stations[gdf_stations.within(gdf.iloc[1].geometry)].id
ams3=gdf_stations[gdf_stations.within(gdf.iloc[2].geometry)].id
ams4=gdf_stations[gdf_stations.within(gdf.iloc[3].geometry)].id

In [ ]:
# recalculer le ratio pour les 4 zones de l'Outaouais ?

def rain_is_significant(row):
    return row[row>1.27].count()

In [ ]:
# rain_sta_ratio = df_stations_precip.apply(lambda row: rain_is_significant(row), 
#                                           axis=1)/df_stations_precip.count(axis=1)

In [ ]:
rain_sta_ratio_asm1 = df_stations_precip[ams1].apply(lambda row: rain_is_significant(row), 
                                               axis=1)/df_stations_precip[ams1].count(axis=1)

rain_sta_ratio_asm2 = df_stations_precip[ams2].apply(lambda row: rain_is_significant(row), 
                                               axis=1)/df_stations_precip[ams2].count(axis=1)

rain_sta_ratio_asm3 = df_stations_precip[ams3].apply(lambda row: rain_is_significant(row), 
                                               axis=1)/df_stations_precip[ams3].count(axis=1)

rain_sta_ratio_asm4 = df_stations_precip[ams4].apply(lambda row: rain_is_significant(row), 
                                               axis=1)/df_stations_precip[ams4].count(axis=1)

rain_sta_ratio = pd.concat([rain_sta_ratio_asm1, rain_sta_ratio_asm2,
                           rain_sta_ratio_asm3, rain_sta_ratio_asm4], axis=1)

In [ ]:
rain_sta_ratio

In [ ]:
df_stations_precip[ams2].count(axis=1).plot()

In [ ]:
ddst = pd.concat([rain_sta_ratio, df_pr_wtr, df_cape], axis=1)
ddst

In [ ]:
rain_sta_ratio.hvplot()

In [ ]:
(df_stations_precip.count(axis=1).rolling(1000).mean()).hvplot()

In [ ]:
rain_sta_ratio.rolling(1000).mean().hvplot()